In [ ]:
import os
import cv2
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import partial
plt.style.use('default')

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

from detection.dataset import VideoDataset, MyConcatDataset, VideoDatasetRNN
from detection.models import TrackNetV2MSE, TrackNetV2NLL, TrackNetV2RNN
from detection.training import train_model
from detection.testing import get_local_maxima

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(device)

%load_ext autoreload
%autoreload 2

# Trajectory analysis with RANSAC on heatmaps

In [ ]:
from detection.testing import get_local_maxima

video_source = '../datasets/dataset_lluis/game5/video.mp4'
# heatmaps_folder = './checkpoints/tracknet_v2_rnn_360_640/phase_3_0/checkpoint_0002_results/heatmaps_test_standard'
# heatmaps_folder = './checkpoints/tracknet_v2_mse_360_640/checkpoint_0027_results/heatmaps_test_standard'
heatmaps_folder = './checkpoints/tracknet_v2_mse_360_640/checkpoint_0027_results/heatmaps_val'
detection_df = pd.read_csv('./checkpoints/tracknet_v2_mse_360_640/checkpoint_0027_results/output_val.csv')
# detection_df = detection_df.loc[detection_df['dataset_id']==1].reset_index()

In [ ]:
def get_frame(video_source, frame_index, w_frame=1280, h_frame=720):
    cap = cv2.VideoCapture(video_source)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index+2)
    ret, frame = cap.read()
    if not ret:
        print('Failed to read frame')
        return None

    return cv2.cvtColor(cv2.resize(frame, (w_frame, h_frame)), cv2.COLOR_BGR2RGB)

def get_heatmap(heatmaps_folder, heatmap_index, w_heatmap=640, h_heatmap=360):
    return cv2.imread(os.path.join(heatmaps_folder, f"{heatmap_index}".zfill(6)+'.png'), cv2.IMREAD_GRAYSCALE)/255

In [ ]:
max_num_candidates = 20
detection_df_part = detection_df[detection_df['dataset_id']==1]

candidates = np.zeros((len(detection_df_part), max_num_candidates, 2))  # candidates array
n_candidates = np.zeros(len(candidates))

for i, df_index in enumerate(detection_df_part.index):
    print(f"{i+1} of {len(detection_df_part)}", end = '\r')
    # frame_index = detection_df['frame_num'][i]
    normalized_heatmap = cv2.imread(os.path.join(heatmaps_folder, f"{df_index}".zfill(6)+'.png'), cv2.IMREAD_GRAYSCALE)/255

    max_heatmap_value = detection_df['max_values'][df_index]
    min_heatmap_value = detection_df['min_values'][df_index]
    heatmap = (normalized_heatmap-min_heatmap_value) * (max_heatmap_value-min_heatmap_value)

    m = get_local_maxima(heatmap, threshold=0.1, sigma=0.5)

    candidates[i, :len(m)] = m[:max_num_candidates]
    n_candidates[i] = min(len(m), max_num_candidates)

print(f"{i+1} of {len(detection_df_part)}")

In [ ]:
k_seed = 5

df_index = detection_df_part.index[k_seed]

frame = get_frame(video_source, detection_df_part['frame_num'][detection_df_part.index[k_seed]])
heatmap = get_heatmap(heatmaps_folder, detection_df_part.index[k_seed])

h_frame, w_frame = frame.shape[:2]

frt = frame.shape[0]/heatmap.shape[0]

plt.imshow(frame)
plt.imshow(cv2.resize(heatmap, (frame.shape[1], frame.shape[0])), cmap='gray', alpha=0.7)
plt.scatter(candidates[k_seed][:,1]*frt, candidates[k_seed][:,0]*frt, s=4, c='r')

plt.show()

## Fit parabolic trajectories

In [ ]:
from trajectories.fitting import *
from trajectories.filtering import *
from trajectories.visualization import *
from trajectories.utils import Graph

In [ ]:
trajectory_info = []

info_keys = ['k_seed','k_min','k_mid','k_max','i_seed','i_min','i_mid','i_max','n_support','iterations']

display_fit = False

t = time.time()

for k in range(len(candidates)):
    k_seed = k

    d_threshold = 5
    seed_radius = 20

    N = 10

    parameters, info, trajectory, supports, cost = fit_trajectory(candidates, n_candidates, k_seed, seed_radius, d_threshold, N)

    if cost is None:
        if display_fit:
            print(f"Seed frame: {k}")
            plt.imshow(frames[k])
            plt.show()
        continue

    if (parameters == 0).all() or info['n_support'] <= 3:
        # exclusion criteria:
        # - points
        # - support set smaller than 5
        continue

    if display_fit:
        # k_min = supports[s,0,]
        print(f"Seed frame: {info['k_seed']}")
        print(f"Used frame for fit: {info['k_min']}, {info['k_mid']}, {info['k_max']}")
        print(f"Used candidate for fit: {info['i_min']}, {info['i_mid']}, {info['i_max']}")
        print(f"support: {supports}")
        print(f"cost = {cost}")
        print(f"Iterations: {info['iterations']}")
        print(f"v = {parameters[0]}")
        print(f"a = {parameters[1]}")
        show_fit(trajectory, candidates,
                *[info[key] for key in list(info.keys())[:-2]],
                background=frames[info['k_min']])
        plt.show()

    d = {key: info[key] for key in info_keys}
    d['v'] = parameters[0]
    d['a'] = parameters[1]
    d['trajectory'] = trajectory
    d['support'] = supports
    d['cost'] = cost
    trajectory_info.append(d)

    print(f"{k+1} of {len(candidates)}", end='\r')

t1 = time.time()
print(f"Execution time: {t1-t}")

plt.show()

## Build trajectory graph

In [ ]:
def extract(trajectory_info, i):
    k_seed = trajectory_info[i]['k_seed']
    trajectory = trajectory_info[i]['trajectory']
    support = trajectory_info[i]['support']
    return trajectory, support, k_seed

In [ ]:
from networkx import DiGraph, dijkstra_path
import networkx as nx

max_diff = N

trajectory_graph = DiGraph()

for i in range(len(trajectory_info)):
    for j in range(i, min(i+max_diff, len(trajectory_info))):
        t1, s1, k1 = extract(trajectory_info, i)
        t2, s2, k2 = extract(trajectory_info, j)
        d = trajectory_distance(t1, s1, k1, t2, s2, k2)

        if d != np.inf and i!=j:
            trajectory_graph.add_edge(k1, k2, weight=d)

In [ ]:
wcc = list(nx.weakly_connected_components(trajectory_graph))
shortest_paths = []

for el in wcc:
    shortest_paths.append(dijkstra_path(trajectory_graph, min(el), max(el)))
    # print(sorted(el))
print(len(shortest_paths))

In [ ]:
for el in wcc:
    print(sorted(el))

In [ ]:
print(shortest_paths[0])

### Visualize subgraph

In [ ]:
G = trajectory_graph.subgraph(range(2399, 2419))
# G = trajectory_graph.subgraph(range(3, 28))

w, h, dpi = 1800, 600, 100
fig, ax = plt.subplots(figsize=(w/dpi, h/dpi), dpi=dpi)

# Create a linear layout
pos = {node: (i, 0) for i, node in enumerate(sorted(G.nodes()))}

# Draw the nodes
nx.draw_networkx_nodes(G, pos, node_color='lightblue', node_size=500, linewidths=2)

def get_index(G, u):
    node_list = list(sorted(G.nodes()))
    return node_list.index(u)

# Draw the curved edges
for u, v, weight in G.edges.data('weight'):
    edge_color = 'r' if weight > 0 else 'k'
    # d = -0.2 * np.log(v-u)  # Control the curvature of the edges
    x = get_index(G, v) - get_index(G, u)
    d = -0.5*(1-np.exp(-0.4*(x-1)))  # Control the curvature of the edges
    d = d if u%2 == 1 else -d
    xs, ys = pos[u]
    xt, yt = pos[v]
    xc = (xs + xt) / 2
    yc = (ys + yt) / 2
    xc += d * (yt - ys)
    yc += d * (xs - xt)
    curve = [(xs, ys), (xc, yc), (xt, yt)]
    nx.draw_networkx_edges(G, pos, edgelist=[(u, v)], edge_color=edge_color, width=1, alpha=1, connectionstyle=f'arc3,rad={d}', arrowstyle='-|>', arrowsize=10)

# Add labels to the nodes
nx.draw_networkx_labels(G, pos, font_color='k')

# Set the x-axis limits to include the nodes
ax.set_xlim(-0.5, len(G.nodes())-0.5)
# Set the y-axis limits
ax.set_ylim([-1, 1])

fig.tight_layout()

# Show the graph
plt.axis('off')
plt.show()

## Visualize on frame

In [ ]:
ti = np.array([t['k_seed'] for t in trajectory_info])

dpi = 50
fig, ax = plt.subplots(figsize=(w_frame/dpi, h_frame/dpi), dpi=dpi)

# colors = ['r', 'g', 'y', 'k']
colors = ['y', 'w']

ax.imshow(get_frame(video_source, detection_df_part['frame_num'][detection_df_part.index[2402]]), zorder=-2)

to_print = []
for i, node in enumerate(shortest_paths[0]):
    if node not in range(2399, 2419):
        continue

    to_print.append(node)
    df_index = np.where(ti==node)[0][0]
    trajectory = trajectory_info[df_index]['trajectory']

    k_min = trajectory_info[df_index]['k_min']
    try:
        next_index = np.where(ti==shortest_paths[0][i+1])[0][0]
        k_max = trajectory_info[next_index]['k_min']
    except IndexError as e:
        k_max = trajectory_info[df_index]['k_max']

    show_fit(trajectory*frt, candidates*frt,
             k_min=k_min,
             k_mid=trajectory_info[df_index]['k_mid'],
             k_max=k_max,
             i_min=trajectory_info[df_index]['i_min'],
             i_mid=trajectory_info[df_index]['i_mid'],
             i_max=trajectory_info[df_index]['i_max'],
             k_seed=trajectory_info[df_index]['k_seed'],
             i_seed=trajectory_info[df_index]['i_seed'],
             ax=ax,
             annotate=True,
             show_fitting_points=True,
             trajectory_color=colors[i%len(colors)])# 'y' if i%2==0 else 'r')
ax.set_xlim(0, w_frame)
ax.set_ylim(h_frame, 0)

print(to_print)
plt.show()

In [ ]:
ti = np.array([t['k_seed'] for t in trajectory_info])

dpi = 50
fig, ax = plt.subplots(figsize=(w_frame/dpi, h_frame/dpi), dpi=dpi)

colors = ['r', 'w', 'g', 'k']
ax.imshow(get_frame(video_source, detection_df_part['frame_num'][detection_df_part.index[99]]), zorder=-2)

# for i, frame in enumerate((107, 115, 123)):
for i, frame in enumerate((100, 101)):
# for i, frame in enumerate((77,)):
    df_index = np.where(ti==frame)[0][0]
    trajectory = trajectory_info[df_index]['trajectory']


    show_fit(trajectory*frt, candidates*frt, *[trajectory_info[df_index][key] for key in info_keys[:-2]],
             ax=ax,
             annotate=True,
             show_fitting_points=True,
             trajectory_color=colors[i%len(colors)])# 'y' if i%2==0 else 'r')
    ax.set_xlim(0, w_frame)
    ax.set_ylim(h_frame, 0)

plt.show()

# Visualize some activations and kernels because why not

In [ ]:
model = TrackNetV2RNN(sequence_length=4)
model.load('checkpoints/tracknet_v2_rnn_360_640/phase_3_0/checkpoint_0002_best.ckpt')
model.eval()
model

In [ ]:
dataset_params = dict(image_size=(360, 640),
                      sequence_length=4,
                      sigma=5,
                      drop_duplicate_frames=False,
                      transform = ToTensor(),
                      target_transform = ToTensor(),
                      grayscale=False)

dataset = VideoDatasetRNN(root="../datasets/prova/", **dataset_params)

In [ ]:
counter = 0

def get_encoding_layer(desired_block=1, subblock=0):
    layers = []
    for i, block in enumerate(model.children()):
        # print(i)
        if i%2 == 1:
            layers.append(block)
        for j, block_element in enumerate(block.children()):
            #print(i, j)
            for k, layer in enumerate(block_element.children()):
                layers.append(layer)
                # print(i, j, k)
                if type(layer) is torch.nn.ReLU and i==2*desired_block and j==subblock:
                    break
            if type(layer) is torch.nn.ReLU and i==2*desired_block and j==subblock:
                break
        if type(layer) is torch.nn.ReLU and i==2*desired_block:
            break
    return layers

def compute_activations(layers, input):
    activation = input.unsqueeze(dim=0)
    with torch.no_grad():
        for l in layers:
            activation = l(activation)

    return activation.squeeze().numpy()

In [ ]:
frames, labels = dataset[50]
frames = frames.to(torch.float32)

In [ ]:
w, h_frame, dpi = 300*2*16/9, 300, 100

fig, axs = plt.subplots(ncols=2, figsize=(w/dpi, h_frame/dpi), dpi=dpi)

axs[0].imshow(frames[-3:].numpy().transpose(1, 2, 0))
axs[0].set_title("Input frame (last in sequence)")

axs[1].imshow(labels[0])
axs[1].set_title("Ground truth")

fig.tight_layout(pad=0.2)
plt.show()

In [ ]:
noise_part = np.linspace(0, 1, 10)
c = []

for n in noise_part:
    with torch.no_grad():
        f = (1-n)*frames + n*torch.randn(frames.shape)
        out = model(f.unsqueeze(dim=0)).squeeze().numpy()
    c.append(out.max())
plt.plot(noise_part, c)

In [ ]:
n = 0.07
with torch.no_grad():
    f = (1-n)*frames + n*torch.randn(frames.shape)
    out = model(f.unsqueeze(dim=0)).squeeze().numpy()
plt.imshow(out)
plt.colorbar()
plt.show()

In [ ]:
frames[:3] = torch.zeros(3, 360, 640)

In [ ]:
block = 2
subblock = 1

activations = compute_activations(get_encoding_layer(block, subblock), frames)
activations.shape

In [ ]:
(dead_activations, ) = np.where(activations.max(axis=(1,2))==0)
print(f"Of {activations.shape[0]} activations, {dead_activations.size} are dead and {activations.shape[0]-dead_activations.size} are not.")

In [ ]:
height_pixels = 1080
top_adjust = 1

w, h_frame, dpi = height_pixels*16/9*top_adjust, height_pixels, 100
fig, axs = plt.subplots(nrows=8, ncols=8, figsize=(w/dpi, h_frame/dpi), dpi=dpi)

i_0 = 0

for k, ax in enumerate(axs.ravel()):
    ax.imshow(activations[k+i_0], cmap='gray')
    # ax.set_title(i)
    ax.set_axis_off()

#fig.suptitle(f"Activations in encoding block {block}, subblock {subblock}")

fig.tight_layout(pad=0.5)
fig.subplots_adjust(top=top_adjust)

#fig.savefig(f"{block}_{subblock}.png")

plt.show()

In [ ]:
model.state_dict().keys()

In [ ]:
dk = 4

kernels = model.state_dict()['vgg_conv1.1.0.weight'].numpy()
biases = model.state_dict()['vgg_conv1.1.0.bias'].numpy()
w, h_frame, dpi = 800, 800, 100
fig, axs = plt.subplots(nrows=8, ncols=8, figsize=(w/dpi, h_frame/dpi), dpi=dpi)

print(kernels.shape)
print(biases[dk])

min_val = kernels[dk].min()
max_val = kernels[dk].max()
print(min_val, max_val)

max_val=max((max_val, -min_val))
min_val=min((-max_val, min_val))

for k, ax in enumerate(axs.ravel()):
    ax.imshow(kernels[dk,k], cmap='RdBu', vmin=min_val, vmax=max_val)
    ax.set_axis_off()

#fig.suptitle(f"Kernel {k}, bias = {biases[k]:.2g}")
fig.tight_layout(pad=0.2)
plt.show()
